In [1]:
# Imports
import IPython
import IPython.display
import pandas as pd
from numpy import mean
from numpy import std
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
import glob
import tensorflow as tf
import math
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

np.random.seed(37151)
tf.random.set_seed(37151)

In [2]:
time = 2.0
steps = int(time * 200)

In [3]:
# Add headers to cvs
# Normalize file names
# Define how much seconds our records have

path = r'Records_Europe_csv'
all_files = glob.glob(path + "/*.csv")

count = 0
for filename in all_files:
    new_file_name = filename.split(".")
    
    df = pd.read_csv(filename, index_col=None, header=None, dtype=float)
    df.set_axis(["time", "aceleration"],axis=1,inplace=True)
    list_time = df['time'].tolist()
    #max_index = np.argwhere(np.array(list_time)>time)[0][0]
    #df.iloc[:(max_index-1)].to_csv('Filtered_Records\\' + new_file_name[5] + '.csv', index = False)
    df.iloc[:steps].to_csv('Filtered_Records\\' + new_file_name[5] + '.csv', index = False)
    count = count + 1
    
print("finished " + str(count) + " files")

finished 0 files


In [4]:
class LSTMClassification():
    def __init__(self):
        pd.options.mode.chained_assignment = None  # default='warn'
        # buscar lista de sismos e respetivas magnitudes para df
        
        df = pd.read_csv('../1-datasets/ESM_flatfile_SA_.csv')
        df.drop(df.columns.difference(['event_id', 'ML']), axis=1, inplace=True)  # limpar sismos sem Magnitude Ml, arredondar ML
        df = df.dropna()
        df['ML'] = df['ML'].apply(lambda x: math.floor(x))       
        df['Values'] = np.nan
        df['Values'] = df['Values'].astype('object')
        
        
        for index, row in df.iterrows():
            try:
                event_id = row['event_id']
                filename = '../Filtered_Records/' + str(event_id) + '.csv'
                df_ts = pd.read_csv(filename, index_col=None, header=0, dtype=np.float32)
                df['Values'][index] = df_ts.to_numpy(dtype='float')     
            except:
                continue
        IPython.display.clear_output()
          
        df = df.dropna()
        df.reset_index(drop=True, inplace=True)
        
        self.df = df.head(10)
        
        print(df.head())
        
        pd.options.mode.chained_assignment = 'warn'  # default='warn'
    
    def prep_features(self, model_output_size=10):
        features = [] # cada linha do array vai ser o ficheiro time acceleration
        labels = [] # mag
        for index, row in self.df.iterrows():
           # try:
                print(row)
                event_id = row['event_id']
                print(event_id)
                filename = '../Filtered_Records/' + str(event_id) + '.csv'
                df_ts = pd.read_csv(filename, index_col=None, header=0, dtype=np.float32)
                print(df_ts)
                features.append(df_ts.to_numpy(dtype='float'))    
            #except:
             #   continue
        t_labels = self.df['ML'].to_numpy()
        
        
        for l in t_labels:
            labels.append(np.eye(model_output_size)[l])
        
        self.features=np.array(features)
        self.labels=np.array(labels)
        
        print(self.features)
        print(self.labels)
    
    def split_dataset(self, split = 0.8):
        self.trainX, self.testX, self.trainY, self.testY = train_test_split(self.features, self.labels, test_size = split, shuffle = True)
    
    def create_model(self, height=400, width=2, model_output_size=10):
        model = Sequential()

        # input shape = num timesteps, num features
        model.add(LSTM(100, input_shape = (height, width)))
        model.add(Dropout(0.5))
        model.add(Dense(100, activation = 'relu'))
        model.add(Dense(model_output_size, activation = 'softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        # Printing the models summary
        #model.summary()
        return model
    
    def evaluate_model(self, model, trainX, trainY, testX, testY, epochs=100, batch_size=20, verbose=0):
        history = model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=verbose)
        loss, accuracy = model.evaluate(testX, testY, batch_size, verbose=verbose)
        return accuracy
    
    def predict_model(self, model, data):
        features = data['Values'].to_numpy().astype(float)
        predictions = model.predict(features)
        return predictions

In [5]:
classifier = LSTMClassification()
classifier.prep_features()

       event_id  ML                                             Values
0  AM-1988-0001   6  [[0.0, 0.0], [0.004999999888241291, -0.0001412...
1  AM-1988-0004   4  [[0.0, 0.0], [0.004999999888241291, 5.31272144...
2  AM-1988-0004   4  [[0.0, 0.0], [0.004999999888241291, 5.31272144...
3  AM-1988-0004   4  [[0.0, 0.0], [0.004999999888241291, 5.31272144...
4  AM-1989-0009   4  [[0.0, 0.0], [0.004999999888241291, -0.0001734...
event_id                                         AM-1988-0001
ML                                                          6
Values      [[0.0, 0.0], [0.004999999888241291, -0.0001412...
Name: 0, dtype: object
AM-1988-0001
      time  aceleration
0    0.000     0.000000
1    0.005    -0.000141
2    0.010    -0.000283
3    0.015    -0.000284
4    0.020    -0.000284
..     ...          ...
395  1.975     0.003461
396  1.980     0.003805
397  1.985     0.004160
398  1.990     0.004529
399  1.995     0.004903

[400 rows x 2 columns]
event_id                                

In [6]:
print(classifier.features.shape)
print(classifier.labels.shape)

classifier.split_dataset()


(10, 400, 2)
(10, 10)


In [7]:
LSTM_magnitude_model = classifier.create_model(height=400, width=2)

In [8]:
classifier.evaluate_model(LSTM_magnitude_model, classifier.trainX,  classifier.trainY,  classifier.testX,  classifier.testY, verbose=1)

Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 2.2693 - accuracy: 0.5000
Epoch 2/100
1/1 [==============================] - 0s 206ms/step - loss: 2.2187 - accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 162ms/step - loss: 1.8779 - accuracy: 1.0000
Epoch 4/100
1/1 [==============================] - 0s 245ms/step - loss: 1.8727 - accuracy: 1.0000
Epoch 5/100
1/1 [==============================] - 0s 190ms/step - loss: 1.9662 - accuracy: 0.5000
Epoch 6/100
1/1 [==============================] - 0s 159ms/step - loss: 1.6911 - accuracy: 1.0000
Epoch 7/100
1/1 [==============================] - 0s 172ms/step - loss: 1.6450 - accuracy: 1.0000
Epoch 8/100
1/1 [==============================] - 0s 332ms/step - loss: 1.3902 - accuracy: 1.0000
Epoch 9/100
1/1 [==============================] - 0s 248ms/step - loss: 1.3568 - accuracy: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 385ms/step - loss: 0.8132 - accuracy: 1.0000
Epoch 11

0.25